<a href="https://colab.research.google.com/github/Lupama2/Maestria-Bunkin/blob/main/Dependencia_laser_Rmax/Simulacion_particulas_equlibrio/simulacion_de_particulas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulación de partículas
versión python cpu

## Importo dependencias

In [40]:
#Importo librerías
import numpy as np
import os

## Defino ctes

In [41]:
#Constantes matemáticas
pi = np.pi

#Ctes físicas
m = 9.11e-31*1e3 #[g]
e = 1.602e-19*(1/3.336641e-10) #[Fr]
c = 299792458*1e2 #[cm/s]
K = 1.380649e-23*(1/1e-7) #constante de Boltzmann [ergio/K], obtenida de NIST


## C.I. (Condiciones Iniciales)

A partir de ahora todas las variables definidas son adimensionales. Las variables que posean dimensiones se declararán con el sufico _dim

In [42]:
#Radio del círculo y velocidad inicial de la partícula
R0_dim = 1e-6 #[cm]
T0_dim = 1000 #[K]

In [43]:
#Calculo las ctes adimensionales
v0_dim = np.sqrt(3*K*T0_dim/m)
alpha = e**2/m/R0_dim/(v0_dim**2)
print(f"Constante adimensional, alpha = {alpha}")

#Radio del círculo y velocidad inicial de la partícula adimensionales
R0 = 1
v0 = 1

print("Radio y velocidad iniciales adimensionales: ", R0, ",\t", v0)

Constante adimensional, alpha = 1.8551552825550501
Radio y velocidad iniciales adimensionales:  1 ,	 1


## Sistema de ecuaciones diferenciales

In [44]:
def f(y, N):
    '''
    Parameters
    ----------
    y_vec (ndarray de dimensión 4N): vector de variables formado por [rx_vec, ry_vec, vx_vec, vy_vec] (en ese orden). Estos son:
        rx_vec (ndarray de dimensión N): vector de posiciones en x de las partículas
        ry_vec (ndarray de dimensión N): vector de posiciones en y de las partículas
        vx_vec (ndarray de dimensión N): vector de velocidades en x de las partículas
        vy_vec (ndarray de dimensión N): vector de velocidades en y de las partículas
    q_vec (ndarray de dimensión N): vector de cargas adimensionales de las partículas
    t (float): tiempo

    Nota:
    N (int): nro de partículas
    '''
    rx_vec = y[:N]
    ry_vec = y[N:2*N]
    vx_vec = y[2*N:3*N]
    vy_vec = y[3*N:4*N]


    #Calculo drdt
    drx_vec = vx_vec
    dry_vec = vy_vec

    #Calculo dvdt
    Ax_matriz = np.tile(rx_vec, (N,1))
    Ay_matriz = np.tile(ry_vec, (N,1))

    Bx_matriz = Ax_matriz.T - Ax_matriz
    By_matriz = Ay_matriz.T - Ay_matriz

    modulo = np.sqrt(Bx_matriz**2 + By_matriz**2)
    modulo = np.where(modulo == 0, 1, modulo) #Para evitar división por cero

    Cx_matriz = Bx_matriz/modulo**3
    Cy_matriz = By_matriz/modulo**3

    dvx_vec = alpha*np.sum(Cx_matriz, axis=1)
    dvy_vec = alpha*np.sum(Cy_matriz, axis=1)
   
    dydt = np.concatenate((drx_vec, dry_vec, dvx_vec, dvy_vec))

    return dydt



In [45]:
from scipy.stats import maxwell

def f_maxwell(N):
    '''
    Devuelve un vector de N velocidades iniciales adimensionales de acuerdo a una distribución de Maxwell-Boltzmann

    Parameters
    ----------
    N (int): nro de partículas
    '''

    factor_Chi_to_v = 1/np.sqrt(m/(K*T0_dim))
    return maxwell.rvs(N)*factor_Chi_to_v/v0_dim


def condiciones_iniciales(N):
    '''
    Devuelve un vector de condiciones iniciales adimensionales

    Parameters
    ----------
    N (int): nro de partículas

    Nota:
    (1) Las posiciones iniciales se distribuyen aleatoriamente en el círculo de radio R0
    (2) Las velocidades iniciales se distribuyen de acuerdo a una distribución de Maxwell-Boltzmann, pero su dirección es aleatoria con distribución uniforme
    '''
    y0 = np.empty(4*N)

    #Posiciones aleatorias
    r0_vec = R0*np.random.rand(N)
    tita0_vec = 2*pi*np.random.rand(N)
    y0[:N] = r0_vec*np.cos(tita0_vec) # = rx0_vec[i]
    y0[N:2*N] = r0_vec*np.sin(tita0_vec) # = ry0_vec[i] 

    #Velocidades de acuerdo a una distribución de Maxwell-Boltzmann 
    v0_vec = f_maxwell(N)
    tita0_vec = 2*pi*np.random.rand(N)
    y0[2*N:3*N] = v0_vec*np.cos(tita0_vec) # = vx0_vec[i]
    y0[3*N:4*N] = v0_vec*np.sin(tita0_vec) # = vy0_vec[i]

    return y0


In [46]:
def distancia_al_origen(r_vec, N):
    '''
    Calcula la distancia al origen de las partículas.

    Parameters
    ----------
    r_vec (ndarray de dimensión 2N): vector de posiciones de las partículas tal que r_vec = [rx_vec, ry_vec]
    N (int): nro de partículas
    '''

    #Opero de forma similar a como lo hice dentro de f
    rx = r_vec[:N]
    ry = r_vec[N:]

    d_vec = np.sqrt(rx**2 + ry**2)

    return d_vec

## Funciones útiles para la evolución

In [47]:
def rebote_duro(rx, ry, vx, vy):
    '''
    Calcula las posiciones y velocidades de una partícula luego del rebote con la pared. Se considera pared dura, es decir, en el choque no solo se invierte la velocidad, sino tmb se refleja su posición.

    Parameters
    ----------
    rx (float): posición x de la partícula
    ry (float): posición y de la partícula
    vx (float): velocidad x de la partícula
    vy (float): velocidad y de la partícula

    Returns
    -------
    rx_new (float): nueva posición x de la partícula
    ry_new (float): nueva posición y de la partícula
    vx_new (float): nueva velocidad x de la partícula
    vy_new (float): nueva velocidad y de la partícula

    Nota:
    Esta función no se emplea durante la evolución. Tiene el problema de que al mover discontinuamente una partícula, la energía potencial del sistema varía de forma discontínua, no conservandose la energía en algunos casos.
    '''

    #Calculo la nueva posición
    factor = -1 + 2*R0/np.sqrt(rx**2 + ry**2)

    rx_new = rx*factor
    ry_new = ry*factor

    #Calculo la nueva velocidad
    
    #Calculo el ángulo del vector [rx,ry]
    tita = np.arctan2(ry, rx)
    vx_new = -vx*np.cos(2*tita) - vy*np.sin(2*tita)
    vy_new = -vx*np.sin(2*tita) + vy*np.cos(2*tita)

    return rx_new, ry_new, vx_new, vy_new

def rebote_blando(rx, ry, vx, vy):
    '''
    Calcula las posiciones y velocidades de una partícula luego del rebote con la pared. Se considera pared blanda, es decir, en el choque solo se invierte la velocidad.

    
    Parameters
    ----------
    rx (float): posición x de la partícula
    ry (float): posición y de la partícula
    vx (float): velocidad x de la partícula
    vy (float): velocidad y de la partícula

    Returns
    -------
    rx_new (float): nueva posición x de la partícula
    ry_new (float): nueva posición y de la partícula
    vx_new (float): nueva velocidad x de la partícula
    vy_new (float): nueva velocidad y de la partícula
    '''

    #Calculo la nueva posición
    rx_new = rx
    ry_new = ry

    #Calculo la nueva velocidad
    
    #Calculo el ángulo del vector [rx,ry]
    tita = np.arctan2(ry, rx)
    vx_new = -vx*np.cos(2*tita) - vy*np.sin(2*tita)
    vy_new = -vx*np.sin(2*tita) + vy*np.cos(2*tita)

    return rx_new, ry_new, vx_new, vy_new


def metodo_Euler(yold, t, dt, N):
    '''
    Método de Euler

    Parameters
    ----------
    yold (ndarray de dimensión 4N): vector de posiciones y velocidades de las partículas en el paso de tiempo anterior
    t (float): tiempo en el paso de tiempo anterior
    dt (float): paso temporal
    N (int): nro de partículas

    Returns
    -------
    ynew (ndarray de dimensión 4N): vector de posiciones y velocidades de las partículas en el paso de tiempo siguiente
    '''
    ynew = yold + dt*f(yold, N)
    return ynew

def metodo_Verlet(yold, t, dt, N):
    '''
    Método de Verlet

    Parameters
    ----------
    yold (ndarray de dimensión 4N): vector de posiciones y velocidades de las partículas en el paso de tiempo anterior
    t (float): tiempo en el paso de tiempo anterior
    dt (float): paso temporal
    N (int): nro de partículas

    Returns
    -------
    ynew (ndarray de dimensión 4N): vector de posiciones y velocidades de las partículas en el paso de tiempo siguiente

    Nota: el esquema numérico está bien definido en el apunte de Física Computacional (FISCOM)
    '''
    
    #Calculo el vector de fuerzas
    dydt = f(yold, N) 
    F_vec = dydt[2*N:]

    #Asigno el vector de posiciones
    r_vec = yold[:2*N]
    #Calculo la posición en el siguiente paso de tiempo
    r_vec_new = r_vec + yold[2*N:4*N]*dt + 1/2*dt**2*F_vec

    #Calculo la fuerza en el siguiente paso de tiempo
    ynew_partial = np.concatenate((r_vec_new, yold[2*N:])) #corrijo solo r_vec

    dydt_new = f(ynew_partial, N)
    F_vec_new = dydt_new[2*N:]

    #Calculo la velocidad en el siguiente paso de tiempo
    v_vec_new = yold[2*N:] + 1/2*dt*(F_vec + F_vec_new)

    return np.concatenate((r_vec_new, v_vec_new))



def avanzo_dt(y, t, dt, N, metodo):
    '''
    Avanza un paso de tiempo el sistema

    Parameters
    ----------
    y (ndarray de dimensión 4N): vector de posiciones y velocidades de las partículas en el paso anterior
    t (float): tiempo en el paso anterior
    dt (float): paso temporal
    N (int): nro de partículas
    metodo (función): método de integración a utilizar

    Returns
    -------
    ynew (ndarray de dimensión 4N): vector de posiciones y velocidades de las partículas en el paso siguiente
    '''
    #Avanzo un paso de tiempo
    ynew = metodo(y, t, dt, N)

    #Verifico si se cumple la condición de choque
    #Calculo la distancia de cada partícula al origen
    d_vec = distancia_al_origen(ynew[0:2*N], N)

    #Determino todos los índices en los que una partícula superó la distancia R0
    indices = np.where(d_vec>R0)[0]
    #Opero sobre las partículas que chocaron
    if len(indices) > 0:
        for indice in indices:
            #Calculo las variables correspondientes
            rx = ynew[indice]
            ry = ynew[indice+N]
            vx = ynew[indice+2*N]
            vy = ynew[indice+3*N]

            #Rebota
            rx_new, ry_new, vx_new, vy_new = rebote_blando(rx, ry, vx, vy)
            #Añado los nuevos datos en ynew de forma ordenada
            ynew[indice] = rx_new
            ynew[indice+N] = ry_new
            ynew[indice+2*N] = vx_new
            ynew[indice+3*N] = vy_new

    return ynew



In [49]:
def savetxt_append(filename, array, append = True):
    '''
    Guarda array en filename en formato txt. Se puede hacer append de los datos o no

    Parameters
    ----------
    filename (str): nombre del archivo con terminación ".txt"
    array (ndarray): array a guardar
    append (bool): si se quiere hacer append de los datos o no   
    '''
    if append == True:
        with open(filename, "ab") as f:
            # f.write(b"\n")
            np.savetxt(f, array, delimiter = " ")
    else:
        with open(filename, "wb") as f:
            # f.write(b"\n")
            np.savetxt(f, array, delimiter = " ")

## Solución numérica

In [50]:
#Hago una simulación
N = 1 #Nro de partículas
dt = 1e-4 #En 1e-1 ya genera problemas. Las partículas salen de la circunferencia. A modo gral, a mayor N, menor dt
n_pasos = 100 #20000*5*10
guardo_cada = 1 #Cada cuántos pasos de tiempo se guardarán los datos



y0 = condiciones_iniciales(N)
y = y0.copy()
t = 0

#Nombres de archivos
files = ["resultados/py_cpu_pos_x.txt", "resultados/py_cpu_pos_y.txt", "resultados/py_cpu_vel_x.txt", "resultados/py_cpu_vel_y.txt"]

#Inicializo
for i in range(n_pasos):
    if i%guardo_cada== 0:
        print(f"t = {round(t,2)}\tEvolución al {round(i/n_pasos*100,2)}%")

    t += dt
    if i % guardo_cada == 0:

        #Guardo datos
        for i in range(4):
            savetxt_append(files[i], y[i*N:(i+1)*N])

    y = avanzo_dt(y, t, dt, N, metodo_Verlet)


t = 0	Evolución al 0.0%
t = 0.0	Evolución al 1.0%
t = 0.0	Evolución al 2.0%
t = 0.0	Evolución al 3.0%
t = 0.0	Evolución al 4.0%
t = 0.0	Evolución al 5.0%
t = 0.0	Evolución al 6.0%
t = 0.0	Evolución al 7.0%
t = 0.0	Evolución al 8.0%
t = 0.0	Evolución al 9.0%
t = 0.0	Evolución al 10.0%
t = 0.0	Evolución al 11.0%
t = 0.0	Evolución al 12.0%
t = 0.0	Evolución al 13.0%
t = 0.0	Evolución al 14.0%
t = 0.0	Evolución al 15.0%
t = 0.0	Evolución al 16.0%
t = 0.0	Evolución al 17.0%
t = 0.0	Evolución al 18.0%
t = 0.0	Evolución al 19.0%
t = 0.0	Evolución al 20.0%
t = 0.0	Evolución al 21.0%
t = 0.0	Evolución al 22.0%
t = 0.0	Evolución al 23.0%
t = 0.0	Evolución al 24.0%
t = 0.0	Evolución al 25.0%
t = 0.0	Evolución al 26.0%
t = 0.0	Evolución al 27.0%
t = 0.0	Evolución al 28.0%
t = 0.0	Evolución al 29.0%
t = 0.0	Evolución al 30.0%
t = 0.0	Evolución al 31.0%
t = 0.0	Evolución al 32.0%
t = 0.0	Evolución al 33.0%
t = 0.0	Evolución al 34.0%
t = 0.0	Evolución al 35.0%
t = 0.0	Evolución al 36.0%
t = 0.0	Evolu

In [51]:
# Continúo la evolución por n2_pasos pasos de tiempo


# n2_pasos = 2*n_pasos #Correspondientes al segundo ciclo de evolución

# pos_x = np.concatenate((pos_x, np.empty([n2_pasos//guardo_cada, N_esperado])))
# pos_y = np.concatenate((pos_y, np.empty([n2_pasos//guardo_cada, N_esperado])))
# vel_x = np.concatenate((vel_x, np.empty([n2_pasos//guardo_cada, N_esperado])))
# vel_y = np.concatenate((vel_y, np.empty([n2_pasos//guardo_cada, N_esperado])))
# q_tot = np.concatenate((q_tot, np.zeros([n2_pasos//guardo_cada, N_esperado])))


# for i in range(n_pasos, n_pasos + n2_pasos):
#     try:
#         y = avanzo_dt(y, q_vec, t, dt, metodo)
#     except ValueError:
#         print(f"Último índice: {i}")
#         break

#     if i%guardo_cada == 0:
#         print(f"t = {round(t,2)}\tEvolución al {round(i/(n_pasos+n2_pasos)*100,2)}%\tN = {np.sum(q_vec)}")


#     t += dt
#     if i%guardo_cada == 0:
#         pos_x[i//guardo_cada] = np.concatenate((y[0:len(y)//4], np.zeros( N_esperado - len(y)//4)))
#         pos_y[i//guardo_cada] = np.concatenate((y[len(y)//4:2*len(y)//4], np.zeros( N_esperado - len(y)//4)))
#         vel_x[i//guardo_cada] = np.concatenate((y[2*len(y)//4:3*len(y)//4], np.zeros( N_esperado - len(y)//4)))
#         vel_y[i//guardo_cada] = np.concatenate((y[3*len(y)//4:4*len(y)//4], np.zeros( N_esperado - len(y)//4)))
#         q_tot[i//guardo_cada] = q_vec

# n_pasos = n2_pasos + n_pasos

# print("N_tot = ", np.sum(q_vec))


In [52]:
#Exporto los tiempos
savetxt_append("resultados/py_cpu_t.txt", np.array([t, dt, n_pasos, guardo_cada]), False)
#Exporto las condiciones iniciales
savetxt_append("resultados/py_cpu_cond_ini.txt", np.array([R0, v0, R0_dim, v0_dim]), False)
